In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame
import datetime
%matplotlib inline

In [2]:
def max_stock(data):
    L = []
    for i in set(data["stock"]):
        counter = list(data["stock"].values).count(i)
        L.append(counter)                                        ###L为各股票出现的次数，即交易天数
    stock_id = list(set(data["stock"]))
    stock_m = stock_id[L.index(max(L))]                           ###选出交易天数最全的一只股票
    return stock_m

In [3]:
def best_perf(data, stock_eg, T, N1, N2, D):                                 ###data为原始数据，T为所要比较的涨跌幅的日期，N1，N2为T之前和T之后的时间跨度，K为最大成交总额及涨跌幅对比数

    data['stock'] = data['stock'].astype(str)                       ###更改stock的数据类型为字符型

    start = int(stock_eg[stock_eg['date'] ==  T].index.values-N1-1)-stock_eg.index[0]                  ###寻找T前N+1天的索引位置
    middle = int(stock_eg[stock_eg['date'] == T].index.values)-1-stock_eg.index[0]             ###寻找T日前一日的索引位置
    end = middle+N2                                                 ###统计涨跌幅最后一日的索引位置
    slc = stock_eg.iloc[[start, middle, middle+1, end],:]           ###选出所需计算的股票信息
    
    init = data[data['date'] == slc['date'].values[0]]
    middle = data[data['date'] == slc['date'].values[1]]
    determ = data[data['date'] == slc['date'].values[2]]
    fina = data[data['date'] == slc['date'].values[3]]
    secl_sum = init.append(middle).sort_index()
    secl_sum = secl_sum.append(determ).sort_index()
    secl_sum = secl_sum.append(fina).sort_index()                   ####将所有在T,T-N1-1,T-1，T-1+N2时有数据的股票信息合成一张表
    
    stock_intersect = list(set(init['stock']).intersection(set(middle['stock'])))
    stock_intersect = list(set(stock_intersect).intersection(set(determ['stock'])))
    stock_intersect = list(set(stock_intersect).intersection(set(fina['stock'])))        ###选出在上述四个时段都有数据的股票
    
    stock_f = pd.DataFrame(columns = ['date', 'stock', 'close', 'value', 'turnover_sum', 'turnover_N1', 'turnover_N2', 'change_N1','change_N2'])  ###建立一个空的dataframe
    
    for k in stock_intersect:
        starter = secl_sum[secl_sum['stock'] == k]
        starter = starter[['date', 'stock', 'close', 'value', 'turnover_sum']]
        turn_o = starter['turnover_sum']
        close = starter['close']
        differ_t = (turn_o.values[1]-turn_o.values[0])/turn_o.values[0]                 ###计算T前N1天的成交金额变化率
        differ_c0 = (close.values[1]-close.values[0])/close.values[0]              ###计算T前N1天的涨跌幅
        differ_t2 = (turn_o.values[3]-turn_o.values[1])/turn_o.values[1]                 ###计算T后N2天的成交金额变化率
        differ_c = (close.values[3]-close.values[1])/close.values[1]              ###计算T后N2天的涨跌幅
        stock_a = starter[starter['date'] == T]
        stock_a.insert(5,'turnover_N1',[differ_t/N1])
        stock_a.insert(6,'turnover_N2',[differ_t2/N2])
        stock_a.insert(7,'change_N1',[differ_c0/N1])                    ###插入N1天的平均日涨跌幅
        stock_a.insert(8,'change_N2',[differ_c/N2])                    ###插入N2天的平均日涨跌幅
        stock_f = stock_f.append(stock_a)
    stock_f = stock_f.sort_index()
    stock_f = stock_f[['date', 'stock', 'close', 'value', 'turnover_N1', 'turnover_N2', 'change_N1', 'change_N2']]
    
    ##stock_Vmin = stock_f.sort_values(by = ['value'], ascending = True).head(K)           ###取出前N1天市值最小的K个股票
    #stock_Tmax = stock_Vmin.sort_values(by = ['turnover_N1'], ascending = False).head(D)     ###取出前N1天成交金额最大的D个股票
    stock_Tmax = stock_f.sort_values(by = ['turnover_N1'], ascending = False).head(D)
    #stock_Cmax = stock_Tmax.sort_values(by = ['change_N1'], ascending = True).head(Z)         ###取出N1天平均日涨跌幅最小的Z个股票
    #stock_best = list(set(stock_Tmax['stock']).intersection(set(stock_Cmax['stock'])))   ###取出涨跌幅及成交金额都是最大K的股票     
    
    #stock_bestinf = pd.DataFrame(columns = ['date','stock','close','turnover_N1','turnover_N2','change_N1','change_N2'])
    #for i in stock_best:
       # infom = stock_Cmax[stock_Cmax['stock'] == i]
        #stock_bestinf = stock_bestinf.append(infom)                                ###获取这几只股票的具体信息
       # stock_bestinf = stock_bestinf.sort_index()
        #stock_bestinf = stock_bestinf[['stock', 'date', 'close', 'value', 'change_N1', 'change_N2', 'turnover_N1', 'turnover_N2']]
    stock_bestinf = stock_Tmax[['stock', 'date', 'close', 'value', 'change_N1', 'change_N2', 'turnover_N1', 'turnover_N2']]
    return stock_bestinf  

In [4]:
def turnover_fun(data, stock_eg, N1, N2, D):
    
    statis = pd.DataFrame(columns = [['date', 'stock']])

    for i in stock_eg['date'].values[N1+1:-N2+1]:
        try:
            stock_bestinf = best_perf(data, stock_eg, i, N1, N2, D)
            stock_bestinf = stock_bestinf[['date', 'stock']]
            statis = statis.append(stock_bestinf, ignore_index=True)
        except:
            continue

    return statis

In [14]:
data = pd.read_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据//2014_4/2014.csv',encoding='gbk',engine='python')

In [15]:
stock_m = max_stock(data)
#stock_m = 1
stock_eg = data[data['stock'] == stock_m]

In [8]:
statis = turnover_fun(data, stock_eg, 3, 5, 100)

In [9]:
statis

,date,stock
0,2014-01-08,300149
1,2014-01-08,2177
2,2014-01-08,2600
3,2014-01-08,600010
4,2014-01-08,2379
5,2014-01-08,2339
6,2014-01-08,2057
7,2014-01-08,2373
8,2014-01-08,620
9,2014-01-08,2224


In [10]:
statis.to_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据/2014_4/2014_stockid.csv',index = False)